- get the Models by X - all of them 
- load them and preidct : from the other pipeline

In [54]:
from utilsml import train_model
from utilsdf import list_files_by_tilenames
from uvars import tilenames_lidar,RES_DPATH
from utilsdf import assign_nulls, fillna,dropnulls_bycol,check_fillnulls, list_files_by_tilenames
from uvars import s1_fnames, s2_fnames,aux_names
import pandas as pd 
from sklearn.model_selection import train_test_split
import os 
from catboost import CatBoostRegressor
import rasterio
import numpy as np

In [55]:
#write_predictions_to_raster(predictions, tile_file, output_raster_path, block_size=256)
import rasterio
import numpy as np

def write_predictions_to_raster(predictions, tile_file, output_raster_path, block_size=256):
    """
    Writes predictions to a new raster file in blocks of specified size using metadata from an existing tile file.
    
    Parameters:
    - predictions (array-like): 1D array of predicted values matching the flattened raster size.
    - tile_file (str): Path to the raster file from which metadata will be read.
    - output_raster_path (str): Path where the new raster file will be saved.
    - block_size (int): The size of the block to write at once. Default is 256.
    
    Returns:
    - None
    """
    # Read metadata from the tile file
    with rasterio.open(tile_file) as src:
        meta = src.meta.copy()
        raster_shape = (src.height, src.width)
        transform = src.transform
        crs = src.crs

    # Reshape predictions to match the raster's dimensions
    try:
        predictions_reshaped = np.array(predictions).reshape(raster_shape)
    except ValueError:
        raise ValueError(f"Predictions array size {len(predictions)} does not match raster dimensions {raster_shape}.")

    # Update metadata for writing a new raster
    meta.update({
        "dtype": rasterio.float32,  # Ensure predictions are stored as float32
        "count": 1,  # Single band
        "compress": "lzw"  # Optional: Add compression
    })

    # Function to write data in blocks
    def write_in_blocks(predictions_reshaped, block_size, dst):
        for i in range(0, predictions_reshaped.shape[0], block_size):
            for j in range(0, predictions_reshaped.shape[1], block_size):
                block = predictions_reshaped[i:i+block_size, j:j+block_size]
                dst.write(block.astype(rasterio.float32), 1, window=rasterio.windows.Window(j, i, block.shape[1], block.shape[0]))
                print(f"Written block ({i}, {j}) to the raster.")

    # Write the new raster in blocks
    with rasterio.open(output_raster_path, "w", **meta) as dst:
        write_in_blocks(predictions_reshaped, block_size, dst)

    print(f"New raster saved to {output_raster_path}")


In [56]:
import os

def find_model_files_by_group(directory, n=3):
    model_files = []
    
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('model.txt'):
                model_files.append(os.path.join(root, file))
    
    # Grouping files into sublists of size n
    grouped_files = [model_files[i:i+n] for i in range(0, len(model_files), n)]
    
    return grouped_files

In [57]:
X = 90
pathx = f"/home/ljp238/Documents/UoE/libe_tabml/output/pa_ctb_traine/{X}"
model_filesx = find_model_files_by_group(pathx, n=3)

In [58]:
model_filesx[2]

['/home/ljp238/Documents/UoE/libe_tabml/output/pa_ctb_traine/90/1000/output/pa_ctb_traine/90/2000/output/pa_ctb_traine/90/5000/N8640000_zdif_90m_catboost_5000_21_model.txt',
 '/home/ljp238/Documents/UoE/libe_tabml/output/pa_ctb_traine/90/1000/output/pa_ctb_traine/90/2000/output/pa_ctb_traine/90/5000/N8640000_zdif_90m_catboost_5000_13_model.txt',
 '/home/ljp238/Documents/UoE/libe_tabml/output/pa_ctb_traine/90/1000/output/pa_ctb_traine/90/2000/output/pa_ctb_traine/90/5000/N8640000_zdif_90m_catboost_5000_43_model.txt']

In [59]:
modelpath = model_filesx[1][0]
model = CatBoostRegressor()
model.load_model(modelpath)

## S: GEN PARQUETS

In [1]:
#from utilsdf import *


In [11]:
import os
import rasterio
import pandas as pd
import numpy as np
from glob import glob
def get_tile_names(tile_files, tilename,X):
    tile_names = [os.path.basename(i).replace('.tif', '') for i in tile_files]
    tile_names = [i.replace(f'_{X}','') for i in tile_names]
    tile_names = [i.replace(f'{tilename}_','').lower() for i in tile_names]
    return tile_names

def get_tile_files(DPATH, X, tilename):
    TILESX = f"{DPATH}{X}"
    tile_dpath = f'{TILESX}/{tilename}'
    fparquet = f'{tile_dpath}/{tilename}_byldem.parquet'
    tile_files = glob(f'{tile_dpath}/*.tif')
    return tile_files, fparquet

def filter_files_by_endingwith(files, var_ending):
    filtered_files = []
    for ending in var_ending:
        matched_files = [f for f in files if f.endswith(ending)]
        #filtered_files.append(matched_files)
        filtered_files.extend(matched_files)
    print(f"Filtered files count: {len(filtered_files)}/{len(files)}")
    return filtered_files

def pathlist2df(vpaths, vnames):
    """
    Reads raster files from a list of paths into a single DataFrame, where each file (or band) is a column.
    Handles NoData values by replacing them with np.nan.

    Parameters:
        vpaths (list): List of file paths to raster datasets.
        vnames (list): List of names to use for the columns in the DataFrame. Must match the number of rasters.

    Returns:
        pd.DataFrame: DataFrame where each column corresponds to a raster or band from the input list.
    """
    if len(vpaths) != len(vnames):
        raise ValueError("The length of vpaths and vnames must be the same.")

    data_dict = {}  # To hold the raster data
    for path, vname in zip(vpaths, vnames):
        with rasterio.open(path) as src:
            #meta = src.meta
            nodata = src.nodata  # Get NoData value
            raster_data = src.read()  # Read all bands
            nbands = src.count

            if nbands > 1:  # Multiband raster
                for band_idx in range(raster_data.shape[0]):
                    column_name = f"{vname}_band{band_idx + 1}"  # Add band suffix
                    band_data = raster_data[band_idx].flatten()  # Flatten the 2D array into 1D
                    band_data = np.where(band_data == nodata, np.nan, band_data)  # Replace NoData with np.nan
                    data_dict[column_name] = band_data
            else:  # Single-band raster
                column_name = vname  # Use vname directly
                raster_data = raster_data.flatten()
                raster_data = np.where(raster_data == nodata, np.nan, raster_data)
                data_dict[column_name] = raster_data

    # Convert to DataFrame
    df = pd.DataFrame(data_dict)
    df = df.astype(np.float32)

    return df

In [12]:
from uvars import RES_DPATH, tilenames_lidar
from uvars import aux_ending90,s1_ending90,s2_ending90,tar_ending90
tar_ending,aux_ending,s1_ending,s2_ending = aux_ending90,s1_ending90,s2_ending90,tar_ending90
vending_all = tar_ending+aux_ending+s1_ending+s2_ending
tilename = tilenames_lidar[0]
DPATH = RES_DPATH
X = 90

In [13]:
tile_files, fparquet = get_tile_files(DPATH, X, tilename)
tile_files = filter_files_by_endingwith(tile_files, vending_all)

Filtered files count: 9/13


In [14]:
tile_files

['/media/ljp238/12TBWolf/RSPROX/TILES90/N09E105/N09E105_EGM08_90.tif',
 '/media/ljp238/12TBWolf/RSPROX/TILES90/N09E105/N09E105_EGM96_90.tif',
 '/media/ljp238/12TBWolf/RSPROX/TILES90/N09E105/N09E105_tdem_HEM_90.tif',
 '/media/ljp238/12TBWolf/RSPROX/TILES90/N09E105/N09E105_multi_S1_90.tif',
 '/media/ljp238/12TBWolf/RSPROX/TILES90/N09E105/N09E105_multi_S2RGB_90.tif',
 '/media/ljp238/12TBWolf/RSPROX/TILES90/N09E105/N09E105_edem_W84_90.tif',
 '/media/ljp238/12TBWolf/RSPROX/TILES90/N09E105/N09E105_tdem_DEM__RIO_0_90.tif',
 '/media/ljp238/12TBWolf/RSPROX/TILES90/N09E105/N09E105_tdem_DEM__Fw_90.tif',
 '/media/ljp238/12TBWolf/RSPROX/TILES90/N09E105/N09E105_multi_DTM_LiDAR_90.tif']

In [ ]:
tile_files

In [8]:

tile_names = get_tile_names(tile_files, tilename,X)
tile_files,tile_names
df = pathlist2df(tile_files, tile_names)

Filtered files count: 9/13


TypeError: expected str, bytes or os.PathLike object, not list

## E: gENERATE PARQUETS

## REMOVE byldem.parquet

In [17]:
from glob import glob 
import shutil
import os 
X = 12#30#90 

prq_pattern = f"/media/ljp238/12TBWolf/RSPROX/TILES{X}/*/*byldem.parquet"
prq_files = glob(prq_pattern)
#for i in prq_files: os.remove(i)

## CHECK byldem.parquet 
- column by file

In [21]:
from glob import glob 
import shutil
import os 
import pandas as pd 
X = 12#30#90 
prq_pattern = f"/media/ljp238/12TBWolf/RSPROX/TILES{X}/*/*byldem.parquet"
prq_files = glob(prq_pattern)

for i, prq in enumerate(prq_files):
    if i > 0: break
    df = pd.read_parquet(prq)
    print(prq)
    print(df.columns.tolist())

In [20]:
df.isna().sum() # the error is in resampling to 30,90 

egm08                      0
egm96                      0
tdem_hem                   0
multi_s1_band1             0
multi_s1_band2             0
multi_s2rgb_band1          0
multi_s2rgb_band2          0
multi_s2rgb_band3          0
multi_s2rgb_band4          0
multi_s2rgb_band5          0
multi_s2rgb_band6          0
multi_s2rgb_band7          0
multi_s2rgb_band8          0
multi_s2rgb_band9          0
multi_s2rgb_band10         0
multi_s2rgb_band11         0
edem_w84                   0
tdem_dem__rio_0            0
tdem_dem__fw          183997
multi_dtm_lidar            0
dtype: int64

In [1]:
# check that data and column match per df 

In [96]:
if X == 90:
    assert len(df) == 1200*1200, ' Grid does not match'

In [95]:
df.shape

(1440000, 20)

# goal train model 90 and predduct before 12pm 

In [90]:


tile_names

['egm08',
 'egm96',
 'tdem_hem',
 'multi_s1',
 'multi_s2rgb',
 'edem_w84',
 'tdem_dem__rio_0',
 'tdem_dem__fw',
 'multi_dtm_lidar']

In [76]:
# everything is wrong here as well

In [ ]:
tfcols = ['edem', 'ldem', 'egm08', 'egm96', 'hem', 'vv', 'vh', 'red', 'green', 'blue', 'nir', 'swir1']
fcols = ['egm08', 'egm96', 'hem', 'vv', 'vh', 'red', 'green', 'blue', 'nir', 'swir1']
tcol = 'edem'

tilenames = tilenames_lidar
fparquet_list,tile_files_list = list_files_by_tilenames(RES_DPATH, X, tilenames)
L = len(fparquet_list)
print(L)
for idx in range(L):
    if idx > 0 : break
    fparquet,tile_files = fparquet_list[idx],tile_files_list[idx]
    print(fparquet)
    df = pd.read_parquet(fparquet_list, columns=tfcols)
    df[fcols] = check_fillnulls(df[fcols]) # only fill fcols -this correct?


6
/media/ljp238/12TBWolf/RSPROX/TILES90/N09E105/N09E105_byldem.parquet


In [61]:
df.shape

(8640000, 12)

In [62]:
df['zdif'] = model.predict(df[fcols])
df['ldem_ml'] = df['edem'].subtract(df['zdif'])

In [63]:
tile_ifile = [i for i in tile_files if 'edem_W84' in i][0]
tile_ofile = tile_ifile.replace('.tif', f'__ML{X}.tif')
tile_ifile,tile_ofile

('/media/ljp238/12TBWolf/RSPROX/TILES90/N09E105/N09E105_edem_W84_90.tif',
 '/media/ljp238/12TBWolf/RSPROX/TILES90/N09E105/N09E105_edem_W84_90__ML90.tif')

In [64]:

write_predictions_to_raster(predictions = np.array(df['ldem_ml']), 
                            tile_file = tile_ifile,
                            output_raster_path =tile_ofile,
                            block_size=256)

ValueError: Predictions array size 8640000 does not match raster dimensions (1200, 1200).

In [66]:
#3600 * 3600 # 30m 12960000 check this for my outing df 
# 1200*1200 # 90m 1440000

In [ ]:
# load data and preprocessing (1min)
# prediction and subtraction (20sec)
# write_predictions_to_raster (1min)